Считывание данных с файла.

Формат вводимых данных в эксель файле:
1. Число сотрудников
2. Число задач
Одна задача -  один назначенный сотрудник.
Один сотрудник - 1+ назначенных проектов.
3. Размерность вектора навыков (размерность вектора одна  на все проекты и всех сотрудников)
4. В каждой ячейке вектора навыков указывается средний и высокий уровни владения навыком для последующего составления правил нечеткого вывода. Каждый навык имеет три уровня владения: low, middle, high. Например, вводятся два числа 10 и 20. Тогда уровень low = все, что меньше 10, уровень middle = от 10 до 20, уровень high = от 20 и выше
5. Для каждого сотрудника заполняется вектор навыков. Получаем матрицу размерности [число сотрудников] * [размерность вектора навыков].
На основе введенных выше данных посредством алгоритма нечеткого вывода составится матрица эффективности. Предполагается, чем больше будет значение целевой функции эффективности, тем быстрее и качественнее будет выполнена задача.
6. Заполняется матрица требуемых навыков для каждой задачи. Размерность матрицы [число задач] * [размерность вектора навыков]
7. Матрица ресурсов по задачам. Размерность матрицы [число сотрудников]*[число задач]
8. Вектор ресурсов по сотрудникам.
9. Количество особей в популяции N
10. Количество итераций без улучшений M
11. Дополнительные условия:
       1) Каждый сотрудник должен быть назначен хотя бы на 1 проект. Данное условие в действительности можно интерпретировать как разгрузку по задачам остальных сотрудников, даже если у них есть доступные ресурсы для выполнения задачи.
   То есть сумма по строке должна быть больше либо равна 1.
       3) На каждую задачу должен быть назначен 1 сотрудник.
   То есть сумма по столбцу должна быть равна 1.

In [1]:
import pandas as pd

def read_data_from_excel(file_path):
    # Чтение Excel-файла
    df = pd.read_excel(file_path, header=None)

    # Количество агентов (сотрудников)
    num_agents = int(df.iloc[0, 0])

    # Количество задач
    num_tasks = int(df.iloc[2, 0])

    # Размерность вектора навыков
    dim_size = int(df.iloc[4, 0])

    # Порог владения средним уровнем навыка
    avg_level = df.iloc[6, 0:dim_size].astype(float).values.tolist()

    # Порог владения высоким уровнем навыка
    high_level = df.iloc[8, 0:dim_size].astype(float).values.tolist()

    # Матрица навыков сотрудника
    agent_skills = df.iloc[10:10+num_agents, 0:dim_size].astype(float).values.tolist()

    # Матрица навыков задач
    task_skills = df.iloc[10+num_agents+1:10+num_agents+1+num_tasks, 0:dim_size].astype(float).values.tolist()

    # Матрица ресурсов по задачам R
    r_matrix = df.iloc[10+num_agents+1+num_tasks+1:10+num_agents+1+num_tasks+1+num_agents, 0:num_tasks].astype(float).values.tolist()

   # Вектор доступных ресурсов B
    b_list = df.iloc[10+num_agents+1+num_tasks+1+num_agents+1:10+num_agents+1+num_tasks+1+num_agents+1+num_agents, 0].astype(float).tolist()

    # Количество особей в популяции N (целое число)
    N = int(df.iloc[10+num_agents+1+num_tasks+1+num_agents+1+num_agents+1, 0])

    # Количество итераций без улучшений M (целое число)
    M = int(df.iloc[10+num_agents+1+num_tasks+1+num_agents+1+num_agents+3, 0])

    return num_agents, num_tasks, agent_skills, task_skills, dim_size, avg_level, high_level, r_matrix, b_list, N, M

Треугольная функция принадлежности.

In [2]:
import numpy as np
import skfuzzy as fuzz
import scipy

def trimff(low_threshold, middle_threshold, high_threshold):
    x = np.linspace(0, 100, 500)
    low_mf = fuzz.trimf(x, [0, low_threshold, middle_threshold])
    middle_mf = fuzz.trimf(x, [low_threshold, middle_threshold, high_threshold])
    high_mf = fuzz.trimf(x, [middle_threshold, high_threshold, 100])

    return low_mf, middle_mf, high_mf, x

Функция оценки  принадлежности каждой компоненты вектора.

In [3]:
def skill_estimation(skill_value, middle_threshold, high_threshold):
    """
    Оценивает уровень владения навыком, используя треугольную функцию принадлежности.
    
    :param skill_value: Значение навыка
    :param low_threshold: Минимальное значение для low. По умолчанию = 0
    :param middle_threshold: Среднее значение для middle
    :param high_threshold: Максимальное значение для high
    :return: Уровень владения навыком (1, 2 или 3)
    """
    low_threshold = 0
    low_mf, middle_mf, high_mf, x = trimff(low_threshold, middle_threshold, high_threshold)
    
    low_degree = fuzz.interp_membership(x, low_mf, skill_value)
    middle_degree = fuzz.interp_membership(x, middle_mf, skill_value)
    high_degree = fuzz.interp_membership(x, high_mf, skill_value)
    
    if low_degree >= middle_degree and low_degree >= high_degree:
        return 1  # Low
    elif middle_degree >= low_degree and middle_degree >= high_degree:
        return 2  # Middle
    else:
        return 3  # High

Запуск алгоритма.

In [4]:
file_path = r"--путь к файлу--
num_agents, num_tasks, agent_skills, task_skills, dim_size, avg_level, high_level, r_matrix, b_list, N, M = read_data_from_excel(file_path)

for index_agent, each_agent in enumerate(agent_skills):
        for index_skill, each_skill in enumerate(each_agent):
            agent_skills[index_agent][index_skill] = skill_estimation(each_skill, avg_level[index_skill], high_level[index_skill])

for index_task, each_task in enumerate(task_skills):
    for index_skill, each_skill in enumerate(each_task):
        task_skills[index_task][index_skill] = skill_estimation(each_skill, avg_level[index_skill], high_level[index_skill])

 
m_matrix = [[None] * num_tasks for _ in range(num_agents)]
for index_task, each_task in enumerate(task_skills):
    for index_agent, each_agent in enumerate(agent_skills):
        m_matrix[index_agent][index_task] = [3 - abs(task_skills[index_task][i] - agent_skills[index_agent][i]) for i in range(dim_size)]


c_matrix = [[None] * num_tasks for _ in range(num_agents)]
for index_agent, each_agent in enumerate(m_matrix):
    for index_task, each_task in enumerate(each_agent):
        c_matrix[index_agent][index_task] = sum(each_task)

c_matrix_np = np.array(c_matrix)
min_value = np.min(c_matrix_np)
max_value = np.max(c_matrix_np)
normalized_matrix = (c_matrix_np - min_value) / (max_value - min_value)
print("Нормализованная матрица:")
print(normalized_matrix)

c_matrix = normalized_matrix

Нормализованная матрица:
[[0.875 0.375 0.375 1.    0.75  0.875 0.5   0.5   0.875 0.625]
 [0.5   0.75  0.75  0.375 0.125 0.25  0.875 0.625 0.5   0.5  ]
 [0.375 0.875 0.625 0.5   0.25  0.375 0.75  0.75  0.625 0.625]
 [0.375 0.625 0.625 0.25  0.    0.375 0.5   0.5   0.625 0.875]
 [0.75  0.75  0.5   0.625 0.375 0.75  0.625 0.625 1.    1.   ]
 [0.5   1.    0.25  0.625 0.375 0.75  0.625 0.625 0.75  0.75 ]
 [0.5   1.    0.5   0.625 0.375 0.5   0.875 0.875 0.75  0.75 ]]


Переходим к генетическому алгоритму.
Определяем целевые функции. Так как данный алгоритм рассчитан на многокритериальный выбор, то целевых функций будет несколько. Например, здесь рассмотрим такие: максимизация эффективности работы, минимизация затрат ресурсов (возможно, максимизация прибыли. Тогда матрица R - это прибыль с проекта, а вектор b - это зарплата сотрудника), минимизация среднего числа проектов на человека. Для удобства будет рассматривать функции минимизации со знаком минус, т.е. решаем задачу максимизации.

In [5]:
# Максимизация эффективности работ
def fitness1(chromosome, c_matrix):
    return round(sum(c_matrix[i][j] for j, i in enumerate(chromosome)), 2)

# Минимизация затрат ресурсов
def fitness2(chromosome, r_matrix):
    return round(-sum(r_matrix[i][j] for j, i in enumerate(chromosome)), 2)

# Минимизация среднего числа проектов на человека
def fitness3(chromosome, num_tasks):
    return round(-num_tasks / len(set(chromosome)), 2)


In [6]:
def unfitness(chromosome, r_matrix, b_list):
    unfitness_value = 0
    agent_resources = np.zeros(len(b_list))
    
    for j, i in enumerate(chromosome):
        agent_resources[i] += r_matrix[i][j]

    for i in range(len(b_list)):
        if agent_resources[i] > b_list[i]:
            unfitness_value += round(agent_resources[i] - b_list[i], 2)

    return round(unfitness_value, 2)


Функция sort_by_values сортирует индексы, указанные в list1 (вектор индексов переменных из рассматриваемого фронта), по значениям в списке values (то есть по значениям вектора целевой функции). Она находит индексы минимальных значений в values и добавляет их в новый список в порядке возрастания этих значений. Если какой-то хромосомы нет в данном фронте, то ставим бесконечность.

In [7]:
import math

def index_of(a, list):
    try:
        return list.index(a)
    except ValueError:
        return -1

def sort_by_values(list1, values):
    sorted_list = []
    values_copy = values[:]
    while len(sorted_list) != len(list1):
        min_index = index_of(min(values_copy), values_copy)
        if min_index in list1:
            sorted_list.append(min_index)
        values_copy[min_index] = math.inf
    return sorted_list

Присваивание рангов Парето-фронтам. 1 ранг присваивается для недоминируемых решений. Далее эти решения отбрасываем и назначаем 2 ранг и т.д., пока множество хромосом не останется пустым. Результатом работы функции является список фронтов, где каждая ячейка содержит хромосомы.

In [ ]:
import math

def fast_non_dominated_sort(values1, values2, values3):
    front = []
    n = [0 for _ in range(len(values1))]
    
    # Функция для проверки доминирования
    def dominates(p, q):
        # Проверяем, доминирует ли решение p решение q по всем 3 целевым функциям
        # Предполагая, что мы максимизируем, доминирование означает:
        # values_p[i] >= values_q[i] для всех i и хотя бы для одного i строго >
        return (values1[p] >= values1[q] and values2[p] >= values2[q] and values3[p] >= values3[q]) and \
                (values1[p] > values1[q] or values2[p] > values2[q] or values3[p] > values3[q])

    def normalize(value, min_value, max_value):
        if min_value == max_value:  # Если все значения одинаковые, нормализуем их к 0.5 (или любому фиксированному значению)
            return 0.5
        return (value - min_value) / (max_value - min_value)

    
    def distance_3d_normalized(p):
        # Нормализуем каждую координату
        norm_value1 = normalize(values1[p], min(values1), max(values1))
        norm_value2 = normalize(values2[p], min(values2), max(values2))
        norm_value3 = normalize(values3[p], min(values3), max(values3))
        
        # Вычисляем расстояние (квадрат расстояния)
        distance_squared = (norm_value1 - 1)**2 + (norm_value2 - 1)**2 + (norm_value3 - 1)**2
        return distance_squared

    
    distances = []  # Список для хранения расстояний
    for i in range(len(n)):
        dist = distance_3d_normalized(i)
        distances.append(dist)

    unique_distances = sorted(set(distances))  # Список уникальных расстояний
    for dist in unique_distances:
        Q = []
        for j in range(len(distances)):
            if distances[j] == dist:
                Q.append(j)
        front.append(Q)


    
    return front


Высчитываем краудинг-расстояние. Данная матрика направлена на анализ "плотности" решений в одном фронте: чем больше расстояние между решениями, тем они разнообразнее. Изначально краудинг-дистанция равна 0. Граничным решениям присваивается значение "inf".

In [9]:

def crowding_distance(values1, values2, values3, front):
    distance = [0 for _ in range(len(front))]
    sorted1 = sort_by_values(front, values1[:])
    sorted2 = sort_by_values(front, values2[:])
    sorted3 = sort_by_values(front, values3[:])


    distance[0] = distance[-1] = float('inf')

    if max(values1) != min(values1):
        for k in range(1, len(front) - 1):
            distance[k] += round((values1[sorted1[k + 1]] - values1[sorted1[k - 1]]) / (max(values1) - min(values1)), 2)

    if max(values2) != min(values2):
        for k in range(1, len(front) - 1):
            distance[k] += round((values2[sorted2[k + 1]] - values2[sorted2[k - 1]]) / (max(values2) - min(values2)), 2)

    if max(values3) != min(values3):
        for k in range(1, len(front) - 1):
            distance[k] += round((values3[sorted3[k + 1]] - values3[sorted3[k - 1]]) / (max(values3) - min(values3)), 2)

    return distance


Кроссовер и мутация хромосом.

In [10]:
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    if random.random() > 0.5:
        child = np.concatenate([parent1[:point], parent2[point:]])
    else:
        child = np.concatenate([parent2[:point], parent1[point:]])
    return child

def mutate(chromosome, num_agents):
    i = random.randint(0, len(chromosome) - 1)
    chromosome[i] = random.randint(0, num_agents - 1)
    return chromosome

Создаем популяцию. Количество особей N задает пользователем.

In [11]:
import numpy as np

def generate_population(N, num_agents, num_tasks):
    population = []
    for _ in range(N):
        while True:
            chromosome = np.random.randint(0, num_agents, num_tasks)
            
            # Проверка, что каждому агенту назначена хотя бы одна задача
            unique_agents, counts = np.unique(chromosome, return_counts=True)
            if len(unique_agents) == num_agents:
                break  # Выход из цикла, если все агенты имеют хотя бы одну задачу
        
        population.append(chromosome)
    
    return population

In [12]:
def ensure_all_agents_assigned(chromosome, num_agents):
    unique_agents, counts = np.unique(chromosome, return_counts=True)
    
    # Проверяем, какие агенты не были назначены
    missing_agents = set(range(num_agents)) - set(unique_agents)
    
    if missing_agents:
        # Если есть недостающие агенты, назначаем им случайные задачи
        for agent in missing_agents:
            # Заменяем случайную задачу, назначенную какому-то другому агенту
            task_idx = np.random.randint(0, len(chromosome))
            chromosome[task_idx] = agent
    
    return chromosome

In [13]:
def binary_tournament_selection(population, non_dominated_sorted_population, crowding_distance_values):
    best1 = None
    best2 = None
    best1_front = float('inf')
    best2_front = float('inf')
    best1_distance = -float('inf')
    best2_distance = -float('inf')
    
    for front_idx, front in enumerate(non_dominated_sorted_population):
        for idx_in_front, idx_in_population in enumerate(front):
            current_chromosome = population[idx_in_population]
            current_distance = crowding_distance_values[front_idx][idx_in_front]
            
            # Проверяем текущую хромосому и сравниваем её с best1 и best2
            if front_idx < best1_front or (front_idx == best1_front and current_distance > best1_distance):
                # Обновляем best1 и сдвигаем старую best1 в best2
                best2 = best1
                best2_front = best1_front
                best2_distance = best1_distance
                
                best1 = current_chromosome
                best1_front = front_idx
                best1_distance = current_distance
            elif front_idx < best2_front or (front_idx == best2_front and current_distance > best2_distance):
                # Обновляем только best2
                best2 = current_chromosome
                best2_front = front_idx
                best2_distance = current_distance
    
    return best1, best2

Основной блок генетического алгоритма. 
1. Иинициализация популяции, N случайных хромосом.

In [14]:
def genetic_algorithm(num_agents, num_tasks, c_matrix, r_matrix, b_list, N, M):
    population = generate_population(N, num_agents, num_tasks)

    gen_no = 0

    while gen_no < M:
        fitness1_values = [round(fitness1(population[i], c_matrix), 2) for i in range(N)]
        fitness2_values = [round(fitness2(population[i], r_matrix), 2) for i in range(N)]
        fitness3_values = [round(fitness3(population[i], num_tasks), 2) for i in range(N)]

        # Сортируем по фронтам, так чтобы каждый фронт доминировался предыдущим и доминировал последующий. Получили список списков, сгруппированный по фронтам
        non_dominated_sorted_population = fast_non_dominated_sort(fitness1_values[:], fitness2_values[:], fitness3_values[:])
        
        # Считаем расстояние Краудинга для каждого фронта
        crowding_distance_values = []
        for i in range(len(non_dominated_sorted_population)):
            crowding_distance_values.append(crowding_distance(fitness1_values[:], fitness2_values[:], fitness3_values[:], non_dominated_sorted_population[i][:]))

        parent1, parent2 = binary_tournament_selection(population, non_dominated_sorted_population, crowding_distance_values)

        child = crossover(parent1, parent2)
        if random.random() < 0.1:
            child = mutate(child, num_agents)


        child_unfitness = unfitness(child, r_matrix, b_list)

        worst_front = non_dominated_sorted_population[-1]

        
        max_unfitness = -float('inf')
        worst_chromosome_idx = None
        for idx in worst_front:
            chrom_unfitness = unfitness(population[idx], r_matrix, b_list)
            if chrom_unfitness > max_unfitness:
                max_unfitness = chrom_unfitness
                worst_chromosome_idx = idx

        if child_unfitness < max_unfitness:
                    population[worst_chromosome_idx] = child
        
        gen_no+=1

    # Собираем результат с учётом фронтов для вывода в виде таблицы
    result = []
    for i, front in enumerate(non_dominated_sorted_population):
        for idx in front:
            f1 = fitness1(population[idx], c_matrix)
            f2 = fitness2(population[idx], r_matrix)
            f3 = fitness3(population[idx], num_tasks)
            result.append({
                "Фронт": i + 1,  # Номер фронта
                "chromosome": population[idx] + 1,  # Если индексация начинается с 0
                "Эфф - ть назначения": f1,
                "Ресурсы": -f2,
                "Ср. нагрузка": -f3
            })
    
    # Преобразуем результат в DataFrame
    df = pd.DataFrame(result)
    
    # Сортируем по фронтам и выводим
    df = df.sort_values(by="Фронт")
    return df


In [15]:
import random

def tournament_selection(population, c_matrix, r_matrix, b_list):
    selected = random.sample(population, 2)
    fit_1 = fitness(selected[0], c_matrix)
    fit_2 = fitness(selected[1], c_matrix)

    if unfitness(selected[0], r_matrix, b_list) == 0 and unfitness(selected[1], r_matrix, b_list) == 0:
        return selected[0] if fit_1 > fit_2 else selected[1]

    if unfitness(selected[0], r_matrix, b_list) == 0:
        return selected[0]

    if unfitness(selected[1], r_matrix, b_list) == 0:
        return selected[1]

    return selected[0] if fit_1 > fit_2 else selected[1]

In [38]:
result = genetic_algorithm(num_agents, num_tasks, c_matrix, r_matrix, b_list, 10, 400)

In [39]:
import pandas as pd

df = pd.DataFrame(result)
    
print(df)

   Фронт                      chromosome  Эфф - ть назначения  Ресурсы  \
0      1  [2, 5, 5, 7, 3, 1, 7, 3, 6, 4]                 6.75     24.0   
1      2  [7, 4, 1, 1, 1, 6, 5, 7, 3, 2]                 6.62     28.0   
2      3  [5, 5, 7, 1, 3, 4, 2, 7, 1, 6]                 7.00     31.0   
3      4  [1, 1, 3, 4, 2, 3, 7, 5, 5, 6]                 5.88     27.0   
4      5  [1, 3, 7, 2, 5, 3, 6, 2, 4, 1]                 5.88     35.0   
5      6  [1, 4, 6, 5, 2, 7, 7, 4, 4, 3]                 5.62     26.0   
6      7  [3, 5, 6, 4, 2, 5, 7, 1, 5, 7]                 5.62     28.0   
7      8  [6, 1, 5, 1, 5, 2, 7, 4, 4, 3]                 5.62     32.0   
8      9  [2, 4, 7, 6, 7, 4, 5, 1, 3, 1]                 5.38     33.0   
9     10  [6, 1, 2, 3, 7, 6, 5, 2, 1, 4]                 6.25     46.0   

   Ср. нагрузка  
0          1.43  
1          1.43  
2          1.43  
3          1.43  
4          1.43  
5          1.43  
6          1.43  
7          1.43  
8          1.43  
9    